In [1]:
global Path
if sc.master[0:5]=='local':
    Path="file:/home/hduser/pythonwork/PythonProject/"
else:
    Path="hdfs://master:9000/user/hduser"

In [4]:
rawUserData = sc.textFile(Path+"/data/u.data")
rawUserData.count()

100000

In [5]:
rawUserData.first()

u'196\t242\t3\t881250949'

In [6]:
from pyspark.mllib.recommendation import Rating

In [7]:
rawRatings = rawUserData.map(lambda line:line.split("\t")[:3])
rawRatings.take(5)

[[u'196', u'242', u'3'],
 [u'186', u'302', u'3'],
 [u'22', u'377', u'1'],
 [u'244', u'51', u'2'],
 [u'166', u'346', u'1']]

In [8]:
ratingsRDD = rawRatings.map(lambda x:(x[0],x[1],x[2]))
ratingsRDD.take(5)

[(u'196', u'242', u'3'),
 (u'186', u'302', u'3'),
 (u'22', u'377', u'1'),
 (u'244', u'51', u'2'),
 (u'166', u'346', u'1')]

In [11]:
numRatings =ratingsRDD.count()
numRatings

100000

In [12]:
numUsers = ratingsRDD.map(lambda x:x[0]).distinct().count()
numUsers

943

In [14]:
numMovies = ratingsRDD.map(lambda x:x[1]).distinct().count()
numMovies

1682

In [15]:
from pyspark.mllib.recommendation import ALS

In [17]:
model= ALS.train(ratingsRDD,10,10,0.01)
print model

In [20]:
model.recommendProducts(196,5)

[Rating(user=196, product=1242, rating=8.434964500570405),
 Rating(user=196, product=1251, rating=8.287903520076153),
 Rating(user=196, product=1176, rating=7.700678605634481),
 Rating(user=196, product=1172, rating=7.415333099995024),
 Rating(user=196, product=621, rating=7.110175516272512)]

In [21]:
model.predict(100,1058)

7.058206334915621

In [23]:
model.recommendUsers(product=200,num=5)

[Rating(user=762, product=200, rating=6.139792722677261),
 Rating(user=808, product=200, rating=6.024028296083621),
 Rating(user=444, product=200, rating=5.768167453064697),
 Rating(user=80, product=200, rating=5.605819597207137),
 Rating(user=282, product=200, rating=5.479047877699429)]

In [27]:
itemRDD = sc.textFile(Path+"/data/u.item")
itemRDD.count()

1682

In [40]:
moviesTitle=itemRDD.map(lambda line:line.split("|")).map(lambda a:(float(a[0]),a[1])).collectAsMap()

In [41]:
itemRDD.first()

u'1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0'

In [43]:
moviesTitle.items()[:5]

[(1.0, u'Toy Story (1995)'),
 (2.0, u'GoldenEye (1995)'),
 (3.0, u'Four Rooms (1995)'),
 (4.0, u'Get Shorty (1995)'),
 (5.0, u'Copycat (1995)')]

In [44]:
moviesTitle[5]

u'Copycat (1995)'

In [53]:
recommendP = model.recommendProducts(190,5)
for p in recommendP:
    print "对用户 "+str(p[0])+" 推荐电影 \n"+str(moviesTitle[p[1]])+"      推荐评分"+str(p[2])

对用户 190 推荐电影 
Dangerous Beauty (1998)      推荐评分5.89441546274
对用户 190 推荐电影 
8 Heads in a Duffel Bag (1997)      推荐评分5.79131584916
对用户 190 推荐电影 
Little Princess, The (1939)      推荐评分5.72651482669
对用户 190 推荐电影 
Scarlet Letter, The (1995)      推荐评分5.41799939811
对用户 190 推荐电影 
Love Affair (1994)      推荐评分5.39664948651
